Third step

Now that the table in UAH is filled in let's prepare the exchange rate for each date.

In [1]:
import pandas as pd

Open sheets separately for easy access.

In [2]:
df = pd.read_excel("ua_bonds.xlsx", sheet_name="Bonds_Info")

In [3]:
df_2 = pd.read_excel("ua_bonds.xlsx", sheet_name="Portfolio_UAH", header=1)

The next solution that comes to my mind: 
    - make a list of all dates, 
    - use  set() to get only unique dates,
    - sort and return to the list,
    - cut the dates before the first purchase.

In [4]:
purchasedates = df_2["Purchase Date"].to_list()
paydates = df["Interest payment dates"].to_list()

In [5]:
dates = paydates + purchasedates
dates = list(set(dates))
dates.sort()
needed_dates = dates[dates.index(purchasedates[0]):]

In [6]:
df_3 = pd.DataFrame(needed_dates, columns=["Dates"])

In [8]:
# Date needs to be str for next step
df_3["Dates"] = df_3["Dates"].astype(str)
df_3["Dates"].values

array(['2023-04-07', '2023-04-08', '2023-04-16', '2023-04-20',
       '2023-04-26', '2023-04-27', '2023-04-30', '2023-05-03',
       '2023-05-05', '2023-05-10', '2023-05-11', '2023-05-17',
       '2023-05-18', '2023-05-24', '2023-05-25', '2023-05-31',
       '2023-06-01', '2023-06-07', '2023-06-08', '2023-06-12',
       '2023-06-28', '2023-07-05', '2023-07-14', '2023-07-17',
       '2023-07-19', '2023-07-23', '2023-07-31', '2023-08-09',
       '2023-08-16', '2023-08-23', '2023-08-30', '2023-08-31',
       '2023-09-06', '2023-09-13', '2023-10-01', '2023-10-04',
       '2023-10-25', '2023-10-31', '2023-11-01', '2023-11-08',
       '2023-11-15', '2023-11-22', '2023-11-29', '2023-12-06',
       '2023-12-20', '2023-12-27', '2024-01-03', '2024-01-17',
       '2024-01-24', '2024-02-07', '2024-02-14', '2024-02-21',
       '2024-02-28', '2024-03-06', '2024-03-13', '2024-04-03',
       '2024-05-08', '2024-05-15', '2024-05-22', '2024-05-29',
       '2024-06-05', '2024-06-19', '2024-06-26', '2024-

In [28]:
import requests

There are dates from future in the list. I can cut them before(# TO DO LIST) =] So now I get the USD/UAH  rate for each date using the free api. Added check to avoid "timetravel" error.

In [35]:
from datetime import datetime

today = str(datetime.today().date())

today

'2023-11-22'

In [36]:
headers = {"app_id": "56774d4e80224f419c0f3d410c187538",
        "base": "USD",
        "symbols": "UAH"}


for date in df_3["Dates"].values:
    if date >= today:
        print("I don't have timetravel machine")
        break
    endpoint = f"https://openexchangerates.org/api/historical/{date}.json"
    response = requests.get(url=endpoint, params=headers)
    df_3.loc[df_3["Dates"] == date, "Rates"] = response.json()["rates"]["UAH"]
    print(f"Rate for {date} was added")

Rate for 2023-04-07 was added
Rate for 2023-04-08 was added
Rate for 2023-04-16 was added
Rate for 2023-04-20 was added
Rate for 2023-04-26 was added
Rate for 2023-04-27 was added
Rate for 2023-04-30 was added
Rate for 2023-05-03 was added
Rate for 2023-05-05 was added
Rate for 2023-05-10 was added
Rate for 2023-05-11 was added
Rate for 2023-05-17 was added
Rate for 2023-05-18 was added
Rate for 2023-05-24 was added
Rate for 2023-05-25 was added
Rate for 2023-05-31 was added
Rate for 2023-06-01 was added
Rate for 2023-06-07 was added
Rate for 2023-06-08 was added
Rate for 2023-06-12 was added
Rate for 2023-06-28 was added
Rate for 2023-07-05 was added
Rate for 2023-07-14 was added
Rate for 2023-07-17 was added
Rate for 2023-07-19 was added
Rate for 2023-07-23 was added
Rate for 2023-07-31 was added
Rate for 2023-08-09 was added
Rate for 2023-08-16 was added
Rate for 2023-08-23 was added
Rate for 2023-08-30 was added
Rate for 2023-08-31 was added
Rate for 2023-09-06 was added
Rate for 2

In [43]:
df_3["Dates"] = pd.to_datetime(df_3["Dates"])

In [44]:
df_3.dtypes

Dates    datetime64[ns]
Rates           float64
dtype: object

The data for the new page is ready. Let's save it in excel.
# TO DO change for "append" instand "replace"

In [45]:
with pd.ExcelWriter("ua_bonds.xlsx", mode="a", engine="openpyxl", if_sheet_exists="replace") as writer:
    df_3.to_excel(writer, sheet_name="USD_UAH", index=0)